# NFL Analysis

An analysis of players from the last 3 seasons of NFL to help with picking for fantasy football.

In [1]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, curdoc
from bokeh.models import ColumnDataSource, HoverTool

In [1]:

nfl_prr_data = pd.read_csv('nfl_pass_rush_receive_raw_data.csv')


#ensure that the date column is of a date-time format.
nfl_prr_data['game_date'] = pd.to_datetime(nfl_prr_data['game_date'])

# This dataset contains 2019-2020, 2020-2021, 2021-2022 data, makes sense to split into 3 datasets initially
se2019_prr = nfl_prr_data.loc[(nfl_prr_data['game_date'] >='2019-09-05') & (nfl_prr_data['game_date'] <='2020-01-19') ]
se2020_prr = nfl_prr_data.loc[(nfl_prr_data['game_date'] >='2020-09-10') & (nfl_prr_data['game_date'] <='2021-02-07') ]
se2021_prr = nfl_prr_data.loc[(nfl_prr_data['game_date'] >='2021-09-09') & (nfl_prr_data['game_date'] <='2022-02-13') ]

print(nfl_prr_data.shape)
print(nfl_prr_data.info())
print(se2019_prr.shape, se2020_prr.shape, se2021_prr.shape)

NameError: name 'pd' is not defined

In [3]:
arodgers19 = se2019_prr[se2019_prr['player'] == 'Aaron Rodgers']
pmahomes19 = se2019_prr[se2019_prr['player'] == 'Patrick Mahomes']
tbrady19 = se2019_prr[se2019_prr['player'] == 'Tom Brady']

print(arodgers19.head())
print(arodgers19.shape)
print(pmahomes19.shape)
print(tbrady19.shape)

           game_id player_id pos         player team  pass_cmp  pass_att  \
0     201909050chi  RodgAa00  QB  Aaron Rodgers  GNB        18        30   
510   201909150gnb  RodgAa00  QB  Aaron Rodgers  GNB        22        34   
904   201909220gnb  RodgAa00  QB  Aaron Rodgers  GNB        17        29   
1156  201909260gnb  RodgAa00  QB  Aaron Rodgers  GNB        34        53   
1554  201910060dal  RodgAa00  QB  Aaron Rodgers  GNB        22        34   

      pass_yds  pass_td  pass_int  ...     OT                       Roof  \
0          203        1         0  ...  False                   outdoors   
510        209        2         0  ...  False                   outdoors   
904        235        1         0  ...  False                   outdoors   
1156       422        2         1  ...  False                   outdoors   
1554       238        0         0  ...  False  retractable roof (closed)   

        Surface  Temperature  Humidity  Wind_Speed  Vegas_Line  \
0         grass     

In [4]:
arodgers19[['game_date']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 6305
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   game_date  18 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 288.0 bytes


In [9]:
#x = arodgers[['game_date']]
#y_1 = arodgers[['pass_td']]

arodgers_source = ColumnDataSource(arodgers19)
tbrady_source = ColumnDataSource(tbrady19)
pmahomes_source = ColumnDataSource(pmahomes19)

hover = HoverTool(tooltips = [ ('Player', '@player'),('Passes Attempted', '@pass_att'), ('Passes Completed', '@{pass_cmp}')])

p = figure(title = 'Pass Attempts vs Completions, 2019 Season',x_axis_type = 'datetime', x_axis_label = 'Date', y_axis_label = 'Passes', tools = [hover])

#Plot for Aaron Rodgers
p.line(x = 'game_date', y = 'pass_cmp', legend_label="Aaron Rodgers Completed",source = arodgers_source, color="red", line_alpha = .5, line_width=3)
circle = p.circle(x = 'game_date', y = 'pass_att', source = arodgers_source, legend_label = 'Aaron Rodgers Attempts',color = 'firebrick',hover_fill_color = 'black', hover_fill_alpha = .5, fill_color = 'red', fill_alpha = .5, size = 8)

#Plot for for Tom Brady
p.line(x = 'game_date', y = 'pass_cmp', legend_label="Tom Brady Completed",source = tbrady_source, color="blue", line_alpha = .5, line_width=3)
p.circle(x = 'game_date', y = 'pass_att', source = tbrady_source, legend_label = 'Tom Brady Attempts', hover_fill_color = 'black', hover_fill_alpha = .5,fill_color = 'blue',fill_alpha = .5, size = 8)


#Plot for Patrick Mahomes
p.line(x = 'game_date', y = 'pass_cmp', legend_label="Patrick Mahomes Completed",source = pmahomes_source, color="orange", line_alpha = .5, line_width=3)
p.circle(x = 'game_date', y = 'pass_att', source = pmahomes_source, legend_label = 'Patrick Mahomes Attempts', hover_fill_color = 'black', hover_fill_alpha = .5,fill_color = 'orange',fill_alpha = .5, size = 8)

output_notebook()

show(p)

Loading BokehJS ...

In [12]:
wide_rec_df = nfl_prr_data[nfl_prr_data['pos'] == 'WR']

top_wr = wide_rec_df[['player', 'team', 'rec_yds']].groupby(['player', 'team']).sum().reset_index()
top_wr = top_wr.sort_values('rec_yds', ascending = False)

print(top_wr.head(10))

            player team  rec_yds
87     Cooper Kupp  LAR     4638
121  Davante Adams  GNB     4445
477    Tyreek Hill  KAN     4123
348     Mike Evans  TAM     3638
476  Tyler Lockett  SEA     3527
97      D.J. Moore  CAR     3525
69    Chris Godwin  TAM     3508
98    D.K. Metcalf  SEA     3485
286   Keenan Allen  LAC     3329
0       A.J. Brown  TEN     3284


In [16]:
top_wr_source = ColumnDataSource(top_wr.head(10))

p1 = figure(title = 'Wide Receivers by Received Yards', x_axis_label = 'yards', y_axis_label = 'Yards', y_range = top_wr['player'].head(10).values.tolist())

p1.hbar(y='player', right='rec_yds', legend_label="Received Yards", color="firebrick", line_color = 'blue', source = top_wr_source)

output_notebook()

show(p1)


Loading BokehJS ...